In [1]:
!wget https://www.kaggle.com/datasets/vipoooool/new-plant-diseases-dataset/download?datasetVersionNumber=2

--2023-09-25 09:03:43--  https://www.kaggle.com/datasets/vipoooool/new-plant-diseases-dataset/download?datasetVersionNumber=2
Resolving www.kaggle.com (www.kaggle.com)... 35.244.233.98
Connecting to www.kaggle.com (www.kaggle.com)|35.244.233.98|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /account/login?titleType=dataset-downloads&showDatasetDownloadSkip=False&messageId=datasetsWelcome&returnUrl=%2Fdatasets%2Fvipoooool%2Fnew-plant-diseases-dataset%2Fversions%2F2%3Fresource%3Ddownload [following]
--2023-09-25 09:03:44--  https://www.kaggle.com/account/login?titleType=dataset-downloads&showDatasetDownloadSkip=False&messageId=datasetsWelcome&returnUrl=%2Fdatasets%2Fvipoooool%2Fnew-plant-diseases-dataset%2Fversions%2F2%3Fresource%3Ddownload
Reusing existing connection to www.kaggle.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘download?datasetVersionNumber=2’

download?datasetVer     [ <=>              

In [3]:
!pwd

/home/ashutosh/work/od/plant_disease


In [4]:
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [6]:
!kaggle datasets download -d vipoooool/new-plant-diseases-dataset

/bin/bash: line 1: kaggle: command not found


In [7]:
import opendatasets as od

In [9]:
od.download('https://www.kaggle.com/datasets/vipoooool/new-plant-diseases-dataset/code?datasetId=78313&sortBy=voteCount')

100%|█████████████████████████████████████████████████████| 2.70G/2.70G [02:59<00:00, 16.2MB/s]


In [1]:
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
import matplotlib.pyplot as plt
tf.compat.v1.set_random_seed(0)
from tensorflow import keras
import numpy as np
np.random.seed(0)
import itertools
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from sklearn.metrics import precision_score, accuracy_score, recall_score, confusion_matrix, ConfusionMatrixDisplay

2023-09-27 09:52:31.254982: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-27 09:52:31.256910: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-27 09:52:31.298293: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-27 09:52:31.299083: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-27 09:52:32.113312: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
train_gen = image_dataset_from_directory(directory="/home/ashutosh/work/od/plant_disease/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train",
                                         image_size=(256, 256))
test_gen = image_dataset_from_directory(directory="/home/ashutosh/work/od/plant_disease/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train",
                                        image_size=(256, 256))

rescale = Rescaling(scale=1.0/255)
train_gen = train_gen.map(lambda image,label:(rescale(image),label))
test_gen  = test_gen.map(lambda image,label:(rescale(image),label))

Found 70295 files belonging to 38 classes.
Found 70295 files belonging to 38 classes.


In [7]:
model = keras.Sequential()

model.add(keras.layers.Conv2D(32,(3,3),activation="relu",padding="same",input_shape=(256,256,3)))
model.add(keras.layers.Conv2D(32,(3,3),activation="relu",padding="same"))
model.add(keras.layers.MaxPooling2D(3,3))

model.add(keras.layers.Conv2D(64,(3,3),activation="relu",padding="same"))
model.add(keras.layers.Conv2D(64,(3,3),activation="relu",padding="same"))
model.add(keras.layers.MaxPooling2D(3,3))

model.add(keras.layers.Conv2D(128,(3,3),activation="relu",padding="same"))
model.add(keras.layers.Conv2D(128,(3,3),activation="relu",padding="same"))
model.add(keras.layers.MaxPooling2D(3,3))

model.add(keras.layers.Conv2D(256,(3,3),activation="relu",padding="same"))
model.add(keras.layers.Conv2D(256,(3,3),activation="relu",padding="same"))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(1568,activation="relu"))
model.add(keras.layers.Dropout(0.5))

model.add(keras.layers.Dense(38,activation="softmax"))

opt = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=opt,loss="sparse_categorical_crossentropy",metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 256, 256, 32)      896       
                                                                 
 conv2d_17 (Conv2D)          (None, 256, 256, 32)      9248      
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 85, 85, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_18 (Conv2D)          (None, 85, 85, 64)        18496     
                                                                 
 conv2d_19 (Conv2D)          (None, 85, 85, 64)        36928     
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 28, 28, 64)        0         
 g2D)                                                 

In [8]:
ep = 1
history = model.fit_generator(train_gen,
          validation_data=test_gen,
          epochs = ep)

2197/2197 [==============================] - 1758s 800ms/step - loss: 1.4425 - accuracy: 0.5764 - val_loss: 0.5286 - val_accuracy: 0.8333


In [9]:
# Save the model to a file
model.save('small_model.h5')

In [9]:
from keras.models import load_model

# Load the saved model
loaded_model = load_model('my_cnn_model.h5')


In [10]:
import tensorflow as tf

# Load the saved Keras model
loaded_model = tf.keras.models.load_model('my_cnn_model.h5')

# Convert the model to TFLite format
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
tflite_model = converter.convert()

# Save the TFLite model to a file
with open('my_cnn_model.tflite', 'wb') as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: /tmp/tmpvkwwe9dx/assets


INFO:tensorflow:Assets written to: /tmp/tmpvkwwe9dx/assets
2023-09-26 06:18:33.798949: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-09-26 06:18:33.798994: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-09-26 06:18:33.801375: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpvkwwe9dx
2023-09-26 06:18:33.804354: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2023-09-26 06:18:33.804377: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /tmp/tmpvkwwe9dx
2023-09-26 06:18:33.812137: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled
2023-09-26 06:18:33.814300: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-09-26 06:18:34.064643: I tensorflow/cc/saved_model/loader.cc:215] Running initializatio

In [5]:
interpreter = tf.lite.Interpreter(model_path='/home/ashutosh/work/od/plant_disease/AllPlantDiseaseCNN.tflite')
interpreter.allocate_tensors()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [6]:
input_data = "/home/ashutosh/work//home/ashutosh/work/od/plant_disease/new-plant-diseases-dataset/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/train/Apple___Cedar_apple_rust/025b2b9a-0ec4-4132-96ac-7f2832d0db4a___FREC_C.Rust 3655_180deg.JPG"

In [8]:
interpreter.set_tensor(0, input_data)  # Set the input tensor
interpreter.invoke()

ValueError: Cannot set tensor: Got value of type STRING but expected type FLOAT32 for input 0, name: serving_default_conv2d_input:0 

In [17]:
input_details = interpreter.get_input_details()
input_details

[{'name': 'serving_default_conv2d_input:0',
  'index': 0,
  'shape': array([  1, 100, 100,   3], dtype=int32),
  'shape_signature': array([ -1, 100, 100,   3], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [2]:
!pwd

/home/ashutosh/work/od/plant_disease


In [10]:
pwd

'/home/ashutosh/work/od/plant_disease'